# Example to visualize and approximate Temperature

In [475]:
import math

import pandas
from pandas.io.json import json_normalize #package for flattening json in pandas df

import folium
from folium.plugins import HeatMap

# library to handle requests
import requests 
import json

import geopy.distance

from datetime import datetime as datetime

FLIX_ALTITUDE = 1975
FLIX_COORDINATES = [769031.4, 155504.5]

POI_COORDINATES = FLIX_COORDINATES
poi_name = "Flix"
poi_altitude = FLIX_ALTITUDE


# https://data.geo.admin.ch/
URL = 'https://data.geo.admin.ch/ch.meteoschweiz.messwerte-lufttemperatur-10min/ch.meteoschweiz.messwerte-lufttemperatur-10min_de.json'

#### Convert the CH LV03 coordinates into WGS84 (LAtitude and Longitude) to place points onto the map

In [476]:
import pyproj

def convert_LV03_to_WGS84(LV03_x, LV03_y):
    
    WGS84 = pyproj.Proj(proj='latlong',datum='WGS84')
    EPSG_21781 = pyproj.Proj(init="EPSG:21781")    
    
    lon, lat = pyproj.transform(EPSG_21781, WGS84, LV03_x, LV03_y)
    
    return lat, lon

#### Fetch data and adjust the dataframe column headings

In [477]:
# Fetch data
data = requests.get(URL).json()
df_meteo = json_normalize(data['features'])

# Select not all columns
df_meteo = df_meteo.filter(["id", "geometry.coordinates", "properties.altitude", "properties.reference_ts", "properties.station_name", "properties.value"])

# Convert altitude to float
df_meteo['properties.altitude'] = df_meteo['properties.altitude'].astype(float)

# Set Column Names
df_meteo.rename(columns={'properties.value':'Temperature', 
                         'properties.station_name':'Station',
                         'properties.altitude':'Altitude',
                         'properties.reference_ts':'Timestamp',
                         'geometry.coordinates':'LV03_coords'
                        }, inplace=True)

print('Shape ', df_meteo.shape)
df_meteo.head()

Shape  (153, 6)


,id,LV03_coords,Altitude,Timestamp,Station,Temperature
0,ARO,"[771030.21, 184825.85]",1880.0,2020-01-01T11:20:00Z,Arosa,5.7
1,RAG,"[756907.4, 209344.82]",498.0,2020-01-01T11:20:00Z,Bad Ragaz,0.9
2,HAI,"[719099.43, 279046.58]",720.0,2020-01-01T11:20:00Z,Salen-Reutenen,-2.3
3,HLL,"[677456.0, 283472.0]",421.0,2020-01-01T11:20:00Z,Hallau,-1.1
4,DEM,"[593269.0, 244543.0]",441.0,2020-01-01T11:20:00Z,Delémont,-3.1


#### Convert LV03 coordinates and add Latitude and Longitude to the dataframe. 
This takes a moment up to a minute

In [478]:
# Add Columns for Latitude and Longitude
df_meteo['Latitude'] = 0
df_meteo['Longitude'] = 0

# Convert coordinates from LV03 to WGS84
for ix in df_meteo.index:
    
    LV03_x, LV03_y = df_meteo.loc[ix, 'LV03_coords']
    latitude, longitude = convert_LV03_to_WGS84(LV03_x, LV03_y)
     
    df_meteo.loc[ix, 'Latitude'] = latitude
    df_meteo.loc[ix, 'Longitude'] = longitude

print('Shape ', df_meteo.shape)
df_meteo.head() 

Shape  (153, 8)


,id,LV03_coords,Altitude,Timestamp,Station,Temperature,Latitude,Longitude
0,ARO,"[771030.21, 184825.85]",1880.0,2020-01-01T11:20:00Z,Arosa,5.7,46.792624,9.679003
1,RAG,"[756907.4, 209344.82]",498.0,2020-01-01T11:20:00Z,Bad Ragaz,0.9,47.016580,9.502541
2,HAI,"[719099.43, 279046.58]",720.0,2020-01-01T11:20:00Z,Salen-Reutenen,-2.3,47.651242,9.023906
3,HLL,"[677456.0, 283472.0]",421.0,2020-01-01T11:20:00Z,Hallau,-1.1,47.697282,8.470463
4,DEM,"[593269.0, 244543.0]",441.0,2020-01-01T11:20:00Z,Delémont,-3.1,47.351702,7.349541


#### Drop locations which are above 60°C and other errornous entries

In [479]:
invalid = df_meteo[df_meteo['Temperature'] > 60].index
df_meteo.drop(invalid , inplace=True)
df_meteo.dropna(inplace=True)

#### Determine the 75% and 25% quantile across all measurment points, and rank the measurements

In [480]:
QT_75 = df_meteo['Temperature'].quantile(.75)
QT_25 = df_meteo['Temperature'].quantile(.25)

df_meteo['Rank'] = 1
df_meteo.loc[df_meteo['Temperature'] >= QT_75, 'Rank'] = 2
df_meteo.loc[df_meteo['Temperature'] <= QT_25, 'Rank'] = 0

print("Temperature threshold at 75%:", QT_75)
print("Temperature threshold at 25%:", QT_25)
df_meteo.head()

Temperature threshold at 75%: 3.3249999999999997
Temperature threshold at 25%: -0.925


,id,LV03_coords,Altitude,Timestamp,Station,Temperature,Latitude,Longitude,Rank
0,ARO,"[771030.21, 184825.85]",1880.0,2020-01-01T11:20:00Z,Arosa,5.7,46.792624,9.679003,2
1,RAG,"[756907.4, 209344.82]",498.0,2020-01-01T11:20:00Z,Bad Ragaz,0.9,47.016580,9.502541,1
2,HAI,"[719099.43, 279046.58]",720.0,2020-01-01T11:20:00Z,Salen-Reutenen,-2.3,47.651242,9.023906,0
3,HLL,"[677456.0, 283472.0]",421.0,2020-01-01T11:20:00Z,Hallau,-1.1,47.697282,8.470463,0
4,DEM,"[593269.0, 244543.0]",441.0,2020-01-01T11:20:00Z,Delémont,-3.1,47.351702,7.349541,0


#### Convert the coordinates of the point of interest into WGS84 (if required)

In [481]:
poi_coordinates_WGS84 = convert_LV03_to_WGS84(POI_COORDINATES[0], POI_COORDINATES[1])
df_meteo.dropna(inplace=True)
df_meteo.head()

,id,LV03_coords,Altitude,Timestamp,Station,Temperature,Latitude,Longitude,Rank
0,ARO,"[771030.21, 184825.85]",1880.0,2020-01-01T11:20:00Z,Arosa,5.7,46.792624,9.679003,2
1,RAG,"[756907.4, 209344.82]",498.0,2020-01-01T11:20:00Z,Bad Ragaz,0.9,47.016580,9.502541,1
2,HAI,"[719099.43, 279046.58]",720.0,2020-01-01T11:20:00Z,Salen-Reutenen,-2.3,47.651242,9.023906,0
3,HLL,"[677456.0, 283472.0]",421.0,2020-01-01T11:20:00Z,Hallau,-1.1,47.697282,8.470463,0
4,DEM,"[593269.0, 244543.0]",441.0,2020-01-01T11:20:00Z,Delémont,-3.1,47.351702,7.349541,0


#### Calculate distances from each point to the point of interest and sort the list along the closest stations

In [482]:
# Add columns for the distance
df_meteo['Distance'] = 0

# Calculate Distance from each point to the point of interest
for ix in df_meteo.index:
    
    station_coords = (df_meteo.loc[ix, 'Latitude'], df_meteo.loc[ix, 'Longitude'])    
    df_meteo.loc[ix,'Distance'] = geopy.distance.distance(poi_coordinates_WGS84, station_coords).km

df_meteo.sort_values(by='Distance', inplace=True)
df_meteo.reset_index(drop=True, inplace=True)
df_meteo.head()

,id,LV03_coords,Altitude,Timestamp,Station,Temperature,Latitude,Longitude,Rank,Distance
0,BIV,"[771279.43, 148114.88]",1858.00,2020-01-01T11:20:00Z,Bivio,-0.6,46.462459,9.668599,1,7.723717
1,PMA,"[760261.0, 160570.0]",2672.00,2020-01-01T11:20:00Z,Piz Martegnas,2.8,46.577188,9.529537,1,10.127843
2,LAT,"[777273.12, 166618.62]",1409.85,2020-01-01T11:20:00Z,Bergün / Latsch,2.4,46.627280,9.753697,1,13.836168
3,SIA,"[778574.95, 144976.5]",1806.00,2020-01-01T11:20:00Z,Segl-Maria,-1.4,46.432335,9.762318,0,14.209248
4,SAM,"[787244.97, 155684.75]",1710.62,2020-01-01T11:20:00Z,Samedan,-7.2,46.526250,9.879399,0,18.213883


#### Linear approximation of the temperature based on the heights and temperatures of the two closest station

In [483]:
t_dif = df_meteo.loc[0,'Temperature'] - df_meteo.loc[1,'Temperature']
a_dif = df_meteo.loc[0,'Altitude'] - df_meteo.loc[1,'Altitude']
ta_slope = t_dif / a_dif

poi_temperature = (poi_altitude - df_meteo.loc[0,'Altitude']) * ta_slope + df_meteo.loc[0,'Temperature']

print('Two closest stations: {} and {}'.format(df_meteo.loc[0,'Station'],df_meteo.loc[1,'Station']))
print('Approximated temperature: {}°C'.format(round(poi_temperature,2)))

Two closest stations: Bivio and Piz Martegnas
Approximated temperature: -0.11°C


#### Display map with all measurement points and highlight the point of interest with a specific marker
Color representate quartiles
- red = 75% --> hottest areas
- green = default  
- blue = 25%  --> coldest areas

In [484]:
# Determine map center
latitude  = df_meteo['Latitude'].mean()
longitude = df_meteo['Longitude'].mean()
colors = ['blue', 'green', 'red']
# Build map
map = folium.Map(location=[latitude, longitude], 
                 tiles='Stamen Terrain',
                 zoom_start=8)

# Setup standard markers
for name, lat, lon, alt, temp, rank in zip(df_meteo['Station'], 
                                           df_meteo['Latitude'],
                                           df_meteo['Longitude'],
                                           df_meteo['Altitude'],
                                           df_meteo['Temperature'],
                                           df_meteo['Rank']):
    
    label = '{} Höhe {}m {}°C'.format(name, int(alt), temp)

    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[rank],
        fill=True,
        fill_color=colors[rank],
        fill_opacity=0.7).add_to(map)  

# Setup special markers for selected locations
fg = folium.FeatureGroup(name='Selected Location')

marker_text = "{} {}°C".format(poi_name, round(poi_temperature,2))

fg.add_child(folium.Marker(location = poi_coordinates_WGS84, 
                           popup = marker_text,
                           icon = folium.Icon(color='pink')))
    
map.add_child(fg)    

map

In [485]:
#convert dataframe to dictionary
dict_meteo = df_meteo.to_dict()

#create a unique nr in order to identify the dict later on
itemnr = datetime.now().strftime("%Y%m%d%H%M")  

#write the dict in a more readable form
meteo_dict = {'item':itemnr,
    'content':[{'name':key,"size":value} for key,value in dict_meteo.items()]}

#convert dict to json object
j = json.dumps(dict_meteo, indent=4)

with open('mydata.json', 'a') as f:  #open in  append mode
    print(j, file=f)
    f.close()